# Face Recognition Service - Demo Notebook

This notebook demonstrates the Face Recognition Service capabilities including:
- Face detection
- Face alignment
- Embedding extraction
- Identity matching
- Gallery management

## 1. Setup and Imports

In [ ]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

# Add FRS to path
sys.path.append('.')

from frs.core.detector import FaceDetector
from frs.core.alignment import FaceAligner
from frs.core.embedding import FaceEmbedding
from frs.core.matcher import FaceMatcher
from frs.database.models import Database

print("✓ Imports successful")

## 2. Initialize Components

In [ ]:
# Initialize detector
print("Loading face detector...")
detector = FaceDetector()
print("✓ Detector loaded")

# Initialize aligner
print("Loading face aligner...")
aligner = FaceAligner()
print("✓ Aligner loaded")

# Initialize embedder
print("Loading embedding model...")
embedder = FaceEmbedding()
print("✓ Embedder loaded")

# Initialize database and matcher
print("Setting up database and matcher...")
db = Database("sqlite:///data/demo.db")
db.create_tables()
matcher = FaceMatcher(db)
print("✓ Database and matcher ready")

print("\n✓ All components initialized successfully!")

## 3. Face Detection Demo

In [ ]:
def visualize_detection(image_path):
    """Detect and visualize faces in an image."""
    # Read image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Failed to load image: {image_path}")
        return
    
    # Detect faces
    start = time.time()
    faces = detector.detect(image)
    elapsed = (time.time() - start) * 1000
    
    print(f"Detected {len(faces)} face(s) in {elapsed:.2f}ms")
    
    # Draw results
    result_img = detector.draw_detections(image, faces)
    
    # Convert BGR to RGB for display
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    
    # Display
    plt.figure(figsize=(12, 8))
    plt.imshow(result_img)
    plt.axis('off')
    plt.title(f'Face Detection Result: {len(faces)} face(s) detected')
    plt.show()
    
    # Print details
    for i, face in enumerate(faces):
        print(f"\nFace {i+1}:")
        print(f"  BBox: {face['bbox']}")
        print(f"  Confidence: {face['confidence']:.3f}")
        print(f"  Quality Score: {face.get('quality_score', 0):.3f}")
    
    return faces

# Example usage (replace with your image path)
# faces = visualize_detection('path/to/your/image.jpg')
print("Function defined. Use: visualize_detection('image_path.jpg')")

## 4. Face Alignment Demo

In [ ]:
def show_alignment(image_path):
    """Show before/after face alignment."""
    image = cv2.imread(str(image_path))
    faces = detector.detect(image)
    
    if len(faces) == 0:
        print("No faces detected")
        return
    
    # Align first face
    face = faces[0]
    aligned = aligner.align(image, face['landmarks'])
    
    # Extract face region
    x1, y1, x2, y2 = face['bbox']
    face_crop = image[y1:y2, x1:x2]
    
    # Display
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Original
    axes[0].imshow(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Face Crop')
    axes[0].axis('off')
    
    # Aligned
    axes[1].imshow(cv2.cvtColor(aligned, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Aligned Face (112x112)')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return aligned

# Example usage
# aligned_face = show_alignment('image.jpg')
print("Function defined. Use: show_alignment('image_path.jpg')")

## 5. Embedding Extraction

In [ ]:
def extract_and_analyze_embedding(image_path):
    """Extract embedding and analyze its properties."""
    image = cv2.imread(str(image_path))
    faces = detector.detect(image)
    
    if len(faces) == 0:
        print("No faces detected")
        return None
    
    # Align and extract embedding
    aligned = aligner.align(image, faces[0]['landmarks'])
    
    start = time.time()
    embedding = embedder.extract(aligned)
    elapsed = (time.time() - start) * 1000
    
    print(f"Embedding extraction: {elapsed:.2f}ms")
    print(f"Embedding shape: {embedding.shape}")
    print(f"Embedding norm (should be ~1.0): {np.linalg.norm(embedding):.4f}")
    print(f"Mean: {embedding.mean():.4f}, Std: {embedding.std():.4f}")
    
    # Visualize embedding distribution
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(embedding, bins=50)
    plt.title('Embedding Value Distribution')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    
    plt.subplot(1, 2, 2)
    plt.plot(embedding)
    plt.title('Embedding Values (512 dimensions)')
    plt.xlabel('Dimension')
    plt.ylabel('Value')
    
    plt.tight_layout()
    plt.show()
    
    return embedding

# Example usage
# emb = extract_and_analyze_embedding('image.jpg')
print("Function defined. Use: extract_and_analyze_embedding('image_path.jpg')")

## 6. Gallery Management

In [ ]:
def add_person_to_gallery(image_path, name, identity_id=None):
    """Add a person to the recognition gallery."""
    import uuid
    
    if identity_id is None:
        identity_id = f"id_{uuid.uuid4().hex[:12]}"
    
    image = cv2.imread(str(image_path))
    faces = detector.detect(image)
    
    if len(faces) == 0:
        print("❌ No face detected")
        return False
    
    if len(faces) > 1:
        print(f"⚠️  Multiple faces detected ({len(faces)}), using first one")
    
    # Extract embedding
    aligned = aligner.align(image, faces[0]['landmarks'])
    embedding = embedder.extract(aligned)
    
    # Add to gallery
    success = matcher.add_identity(
        identity_id=identity_id,
        name=name,
        embedding=embedding,
        image_path=str(image_path)
    )
    
    if success:
        print(f"✓ Added {name} to gallery (ID: {identity_id})")
        print(f"  Gallery size: {len(matcher.get_all_identities())}")
    else:
        print(f"❌ Failed to add {name}")
    
    return success

# Example usage
# add_person_to_gallery('person1.jpg', 'John Doe')
# add_person_to_gallery('person2.jpg', 'Jane Smith')
print("Function defined. Use: add_person_to_gallery('image.jpg', 'Person Name')")

## 7. Face Recognition

In [ ]:
def recognize_faces(image_path, top_k=5):
    """Recognize faces in an image."""
    image = cv2.imread(str(image_path))
    
    # Detect faces
    faces = detector.detect(image)
    print(f"Detected {len(faces)} face(s)")
    
    if len(faces) == 0:
        return
    
    # Recognize each face
    result_img = image.copy()
    
    for i, face in enumerate(faces):
        # Extract embedding
        aligned = aligner.align(image, face['landmarks'])
        embedding = embedder.extract(aligned)
        
        # Match
        matches = matcher.match(embedding)
        
        # Draw results
        x1, y1, x2, y2 = face['bbox']
        
        if matches:
            best_match = matches[0]
            name = best_match['name']
            confidence = best_match['confidence']
            color = (0, 255, 0) if confidence > 0.6 else (0, 165, 255)
            label = f"{name} ({confidence:.2f})"
        else:
            color = (0, 0, 255)
            label = "Unknown"
        
        # Draw box and label
        cv2.rectangle(result_img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(result_img, label, (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        # Print matches
        print(f"\nFace {i+1}:")
        if matches:
            print(f"  Top matches:")
            for j, match in enumerate(matches[:top_k]):
                print(f"    {j+1}. {match['name']}: {match['confidence']:.3f}")
        else:
            print("  No matches found")
    
    # Display
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 8))
    plt.imshow(result_img)
    plt.axis('off')
    plt.title('Face Recognition Results')
    plt.show()

# Example usage
# recognize_faces('test_image.jpg')
print("Function defined. Use: recognize_faces('image_path.jpg')")

## 8. Performance Benchmarking

In [ ]:
def benchmark_pipeline(image_path, num_iterations=10):
    """Benchmark the end-to-end pipeline."""
    image = cv2.imread(str(image_path))
    
    times = {
        'detection': [],
        'alignment': [],
        'embedding': [],
        'matching': [],
        'total': []
    }
    
    print(f"Running {num_iterations} iterations...")
    
    for i in range(num_iterations):
        start_total = time.time()
        
        # Detection
        start = time.time()
        faces = detector.detect(image)
        times['detection'].append((time.time() - start) * 1000)
        
        if len(faces) == 0:
            continue
        
        # Alignment
        start = time.time()
        aligned = aligner.align(image, faces[0]['landmarks'])
        times['alignment'].append((time.time() - start) * 1000)
        
        # Embedding
        start = time.time()
        embedding = embedder.extract(aligned)
        times['embedding'].append((time.time() - start) * 1000)
        
        # Matching
        start = time.time()
        _ = matcher.match(embedding)
        times['matching'].append((time.time() - start) * 1000)
        
        times['total'].append((time.time() - start_total) * 1000)
    
    # Calculate statistics
    print("\n" + "="*60)
    print("PERFORMANCE BENCHMARK RESULTS")
    print("="*60)
    
    for component, timings in times.items():
        if timings:
            avg = np.mean(timings)
            std = np.std(timings)
            min_t = np.min(timings)
            max_t = np.max(timings)
            fps = 1000 / avg if avg > 0 else 0
            
            print(f"\n{component.upper()}:")
            print(f"  Average: {avg:.2f}ms ± {std:.2f}ms")
            print(f"  Min/Max: {min_t:.2f}ms / {max_t:.2f}ms")
            print(f"  Throughput: {fps:.2f} FPS")
    
    # Visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart
    components = ['Detection', 'Alignment', 'Embedding', 'Matching']
    avg_times = [np.mean(times[k]) for k in ['detection', 'alignment', 'embedding', 'matching']]
    
    ax1.bar(components, avg_times, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax1.set_ylabel('Latency (ms)')
    ax1.set_title('Average Latency by Component')
    ax1.grid(axis='y', alpha=0.3)
    
    # Pie chart
    ax2.pie(avg_times, labels=components, autopct='%1.1f%%', startangle=90,
           colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax2.set_title('Latency Distribution')
    
    plt.tight_layout()
    plt.show()
    
    return times

# Example usage
# benchmark_results = benchmark_pipeline('image.jpg', num_iterations=20)
print("Function defined. Use: benchmark_pipeline('image.jpg', num_iterations=20)")

## 9. Complete End-to-End Example

In [ ]:
# Example workflow:
# 1. Add people to gallery
# add_person_to_gallery('data/person1.jpg', 'Person 1')
# add_person_to_gallery('data/person2.jpg', 'Person 2')

# 2. List gallery
identities = matcher.get_all_identities()
print(f"Gallery contains {len(identities)} identities:")
for identity in identities:
    print(f"  - {identity['name']} (ID: {identity['identity_id']})")

# 3. Recognize faces in new image
# recognize_faces('data/test_image.jpg')

# 4. Benchmark performance
# results = benchmark_pipeline('data/test_image.jpg', num_iterations=10)

print("\n✓ Ready to run examples!")
print("\nQuick start:")
print("1. add_person_to_gallery('your_image.jpg', 'Name')")
print("2. recognize_faces('test_image.jpg')")
print("3. benchmark_pipeline('test_image.jpg')")